#Imports

In [346]:
#Clone the transformers repo into the notebook
!git clone https://github.com/huggingface/transformers

fatal: destination path 'transformers' already exists and is not an empty directory.


In [347]:
import os
import torch
os.chdir("transformers")
os.chdir("./examples/language-modeling")
!ls

README.md	  run_clm.py	   run_mlm_no_trainer.py  run_plm.py
requirements.txt  run_mlm_flax.py  run_mlm.py		  transformers


In [348]:
!pip install -r requirements.txt

In [349]:
!pip install pyarrow --upgrade

Requirement already up-to-date: pyarrow in /usr/local/lib/python3.7/dist-packages (3.0.0)


In [350]:
import os
os.chdir("/content/transformers/examples/")
os.chdir("./language-modeling")

In [351]:
cuda0 = torch.device('cuda:0')

In [352]:
# Need to install latest transformer packages from github so the scripts will run correctly
! pip install git+git://github.com/huggingface/transformers/

  Cloning git://github.com/huggingface/transformers/ to /tmp/pip-req-build-n6za6zu6
  Running command git clone -q git://github.com/huggingface/transformers/ /tmp/pip-req-build-n6za6zu6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.5.0.dev0-cp37-none-any.whl size=2064484 sha256=c36e8437b8cbe7a215d3a36f45b01f1122ff742520a7a385dcd532d732657cdf
  Stored in directory: /tmp/pip-ephem-wheel-cache-g5es_pdv/wheels/dc/e5/1e/3a2977a646558fca07585cadcf56aa4a910e995ba945961c4e
Successfully built transformers


#Data Processing

In [373]:
from sklearn.model_selection import train_test_split

with open('/content/Lectures_1.csv', 'r') as data:
  dataset = [x.strip() for x in data.readlines()]

  
train, eval = train_test_split(dataset, train_size=.9, random_state=2020)
print("training size:" + str(len(train)))
print("Evaluation size: " + str(len(eval)))

with open('/content/train_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(train))
  file_handle.close()

with open('/content/eval_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(eval))

training size:80505
Evaluation size: 8946


#Train

In [376]:
!python run_clm.py \
--model_type gpt2  \
--model_name_or_path distilgpt2 \
--train_file "/content/train_tmp.txt" \
--do_train \
--validation_file "/content/eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 5 \
--fp16 \
--output_dir="/content/test_model"

2021-04-04 04:38:28.687334: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
04/04/2021 04:38:30 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
04/04/2021 04:38:30 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=/content/test_model, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/Apr04_04-38-30_e529a484f387, logging_strategy=IntervalStrategy.STEPS, logging_first_ste

In [378]:
from transformers import GPT2LMHeadModel
from transformers import GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained("/content/test_model")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


#Generate Test

In [379]:
input_ids = tokenizer.encode("The reason", return_tensors='tf')

In [380]:
# the tf tensor object
input_ids[0]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 464, 1738], dtype=int32)>

In [381]:
generated_text_samples = model.generate(
    input_ids, 
    max_length=200,  
    num_return_sequences=5,
    temperature=.95,
    do_sample=True,
    top_k=100,
)

AttributeError: ignored

In [377]:
import numpy as np

tokenizer = GPT2Tokenizer.from_pretrained("/content/test_model")
model = GPT2LMHeadModel.from_pretrained('gpt2')

model = model.to('cuda')

def generate(input_str, length=250, n=5):
  cur_ids = torch.tensor(tokenizer.encode(input_str)).unsqueeze(0).long().to(device)
  model.eval()
  with torch.no_grad():
    for i in range(length):
      outputs = model(cur_ids[:, -1024:], labels=cur_ids[:, -1024:])
      loss, logits = outputs[:2]
      softmax_logits = torch.softmax(logits[0,-1], dim=0)
      next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n)
      cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim=1)
    output_list = list(cur_ids.squeeze().to('cpu').numpy())
    output_text = tokenizer.decode(output_list)
    return output_text

def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

generated_text = generate(" = Physics = \n")
print(generated_text)

 = Physics = 

{

// This is a very simple function to use, just use the function

// in the constructor

if (! is_null ())

{

return nullptr ;

}

// This is a very simple function to call in the constructor

if (! is_null () &&! is_null () )

{

return nullptr ;

}

// This is a very simple function to call in the constructor

if (! is_null () &&! is_null (). empty ())

{

return nullptr ;

}

return is_null ()? nullptr : nullptr ;

}

// This is a function to call in the constructor

// in a function

if (! is_null () && is_null () && is_null () && is_null ( is_null (), nullptr ))

{

return nullptr ;

}

// This is a function to call in the constructor

// in an object

if (! is_null ())

{

return nullptr ;

}

else


In [ ]:
# Print output for each sequence generated above
for i, beam in enumerate(generated_text_samples):
  print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=True)))
  print()


In [ ]:
!zip -r /content/file.zip /content/Folder_To_Zip
from google.colab import files
files.download("/content/file.zip")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')